In [1]:
"""Create parameters.py from a .inp and load it"""
# This cell is facultative, you can use an existing parameters.py

# Directory for .inp file:
path = 'INP file examples//chem_Otomo2018_without_O_reactions.inp'#'INP file examples//chem_Otomo2018.inp'#'INP file examples//chemkin_AR_HE.inp'

# import libraries:
import importlib   # for reloading your own files
# my own files:
import inp_data_extractor as inp
importlib.reload(inp)

# create parameters.py
inp.extract(path)

# load parameters.py
import parameters as par
importlib.reload(par)
print(par.model)

Note, lambda value for specie 'NO' is not in data.py: 0.0 is used
Note, lambda value for specie 'H' is not in data.py: 0.0 is used
Note, lambda value for specie 'O' is not in data.py: 0.0 is used
Note, lambda value for specie 'OH' is not in data.py: 0.0 is used
Note, lambda value for specie 'HO2' is not in data.py: 0.0 is used
Note, lambda value for specie 'NH2' is not in data.py: 0.0 is used
Note, lambda value for specie 'NH' is not in data.py: 0.0 is used
Note, lambda value for specie 'N' is not in data.py: 0.0 is used
Note, lambda value for specie 'NNH' is not in data.py: 0.0 is used
Note, lambda value for specie 'NH2OH' is not in data.py: 0.0 is used
Note, lambda value for specie 'H2NO' is not in data.py: 0.0 is used
Note, lambda value for specie 'HNOH' is not in data.py: 0.0 is used
Note, lambda value for specie 'HNO' is not in data.py: 0.0 is used
Note, lambda value for specie 'HON' is not in data.py: 0.0 is used
Note, lambda value for specie 'HONO' is not in data.py: 0.0 is used

In [2]:
"""Libraries"""

# for plotting:
%matplotlib inline    
#If you run this code on the HDS department CPU you should comment the previous line! This line is needed only if you run it in Jupyter IPython.
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})

import numpy as np   # matrices, math
import time   # runtime measurement
from multiprocessing import Pool, cpu_count   # multithreading
import importlib   # reload changes you made

# my own file:
import full_bubble_model as de    # full bubble model
importlib.reload(de)   # reload changes you made

model: INP file examples//chem_Otomo2018_without_O_reactions
Enable heat transfer: True, enable evaporation: False, enable reactions: True
model: INP file examples//chem_Otomo2018_without_O_reactions
Enable heat transfer: True, enable evaporation: False, enable reactions: True


<module 'full_bubble_model' from 'C:\\Users\\mrkf9\\Documents\\KozakAron\\python\\ammonia_TDK_simulation\\Bubble_dynamics_simulation\\full_bubble_model.py'>

In [40]:
ranges = de.dotdict(dict(
    # Equilibrium radius [um --> m]
    R_E = [1.0e-6*x for x in np.concatenate(([1.0,5.0,10.0,15.0,20.0,30.0,40.0,50.0,60.0,70.0,80.0,90.0],np.linspace(100,900,num=41)))], #0.0007329104, #[1.0, 2.0, 3.0, 4.0, 5.0, 7.5, 10.0, 15.0, 20.0, 25.0]],
    # R_star / R_E [-]
    ratio = [x for x in np.linspace(1.2,10,num=45)], #[2.0, 3.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 40.0, 50.0],
    # Ambient pressure [Pa]
    P_amb = [1.01325e5*x for x in [1.0]],#np.linspace (-0.2,1.1,num=27)], #1.01325e5*x for x in np.linspace(-0.2,0.2,num=101) #[1e5*x for x in [1.0, 10.0, 25.0, 50.0]],
    # Accommodation coeff. for evaporation [-]
    alfa_M = [0.35],
    # Ambient temperature [°C --> K]
    T_inf = [273.15+x for x in [20.0]],
    # Surface tension modifier [-]
    surfactant = [1.0], #np.linspace(0.0,1.0,num=6),
    # Molar fractions of species in initial bubble (H2 and N2) [-]
    fractions = [[x,1.0-x] for x in np.linspace(0.4,0.85,num=10)], #[0.7195518738812899,0.28044812611870934], #[[0.40, 0.60],[0.50, 0.50],[0.6, 0.4],[0.7,0.3],[0.75, 0.25],[0.8, 0.2],],
    #Exciting frequency 1 [Hz]
    freq1 = [2.0e4], 
    #Exciting frequency 2 [Hz]
    freq2 = [1.0e5], 
    #Exciting pressure amplitude 1 [Pa]
    pA1 = [0.0e5], 
    #Exciting pressure amplitude 2 [Pa]
    pA2 = [0.0e5], 
    #Initial phase angle between the exciting pressure waves [rad]
    theta_phase = [0.1],
    #Dynamic viscosity of the liquid [Pa*s]
    mu_L = [0.001017649], #[0.001 * x for x in np.logspace(np.log10(1.0),np.log10(100.0),num=3)],
    #Sound velocity in the liquid [m/s]
    c_L = [1483.0], #[1483.0 * x for x  in np.linspace(0.0,2.0,num=5)],  
))

# print total combinations:
for key in ranges:
    print(f'{key}: {len(ranges[key])}')
print(f'total combinations: {np.prod([len(ranges[key]) for key in ranges])}')

R_E: 53
ratio: 45
P_amb: 1
alfa_M: 1
T_inf: 1
surfactant: 1
fractions: 10
freq1: 1
freq2: 1
pA1: 1
pA2: 1
theta_phase: 1
mu_L: 1
c_L: 1
total combinations: 23850


In [5]:
"""Get all combinations"""
# Make a list, with one dictionary for eachy parameter combinations

cpars = []
ID = 1
for R_E in ranges.R_E:
    for ratio in ranges.ratio:
        for P_amb in ranges.P_amb:
            for alfa_M in ranges.alfa_M:
                for T_inf in ranges.T_inf:
                    for surfactant in ranges.surfactant:
                        for fractions in ranges.fractions:
                            for mu_L in ranges.mu_L:
                                for c_L in ranges.c_L:
                                    cpar = dict(
                                        ID=ID,
                                        R_E=R_E, # [m]
                                        ratio=ratio, # [-]
                                        P_amb=P_amb, # [Pa]
                                        alfa_M=alfa_M, # [-]
                                        T_inf=T_inf, # [m]
                                        surfactant=surfactant, # [-]
                                        gases=[de.par.index['H2'], de.par.index['N2']], # indexes of species in initial bubble
                                        fractions=fractions, # molar fractions of species in initial bubble
                                        freq1=2.0e4, # [Hz]
                                        freq2=1.0e5, # [Hz]
                                        pA1=0.0e5, # [Pa]
                                        pA2=0.0e5, # [Pa]
                                        theta_phase=0.1, # [rad]
                                        mu_L=mu_L, #[Pa*s]
                                        c_L=c_L, #[m/s]
                                    )
                                    cpar['P_v'] = de.VapourPressure(cpar['T_inf']) # [Pa]
                                    #cpar['mu_L'] = de.Viscosity(cpar['T_inf']) # [Pa*s]
                                    cpars.append(cpar)
                                    ID += 1
                            
# Create input dictionary for de.simulate(), a list of dictionaries with cpar and other arguments
kwargs_list = [dict(cpar=cpar, t_int=np.array([0.0, 1.0e-3]), LSODA_timeout=300, Radau_timeout=30) for cpar in cpars]

In [ ]:
"""Parameter study, multithread"""
# Runs each combinations in cpars, and saves the results into CSV files
# use Pool(processes=cpu_count()-1) to limit number of threads being used.
# use pool.imap(...) instead of pool.imap_unordered(...) to preserve order in which cpars was made

max_lines = 11000    # maximum length of a CSV
best_energy = 1.0e30

start = time.time()
file = de.Make_dir('test 1')
file.new_file()
with Pool(processes=cpu_count(), maxtasksperchild=100) as pool:
    results = pool.imap_unordered(de.simulate, kwargs_list)

    for data in results:
      # save results:
        if file.lines > max_lines:
            file.close()
            file.new_file()
        data = de.dotdict(data)
        file.write_line(data)
      # print stuff:
        if data.energy_efficiency>0 and data.energy_efficiency<best_energy:
            best_energy = data.energy_efficiency
        print(f'index: {data.ID}/{len(cpars)},   error_code: {data.error_code},   steps: {data.steps},   runtime: {data.elapsed_time: .2f} [s]   |   '+
              f'R_E={1.0e6*data.R_E: .2f} [um], ratio={data.ratio: .2f} [-], P_amb={1.0e-5*data.P_amb: .6f} [bar], initial H2={100.0*data.fractions[0]} %, '+
              f'alfa_M={data.alfa_M: .2f} [-], T_inf={data.T_inf-273.15: .2f} [°C], surfactant={100.0*data.surfactant: .0f} [%], freq1 ={data.freq1: .0f} [Hz], '+
              f'freq2 ={data.freq2: .0f} [Hz], pA1 ={data.pA1: .0f} [Hz], pA2 ={data.pA2: .0f} [Hz], theta_phase={data.theta_phase: .2f} [rad]   |   '+
              f'mu_L ={data.mu_L: .5f} [Pa*s], c_L ={data.c_L: .3f} [m/s]   |   '+
              f'ammonia production: {data.energy_efficiency: .1f} [MJ/kg] (best: {best_energy: .1f} [MJ/kg])', end='\r')
            
file.close()
end = time.time()
elapsed = end - start
print(f'\n\nDONE')
print(f'total time: {(elapsed-(elapsed % 3600))/3600: .0f} hours {((elapsed % 3600) / 60): .0f} mins')
print(f'            {elapsed: .2f} [s]   ({(elapsed / len(cpars)): .2f} [s/run])')

Folder already exists with 8 csv in it
